## One dictionary to rule them all
 
Had previously created team_dictionary for specific years. Need one main one to work with merged data and incorporate Elo table.

In [1]:
import pandas as pd
import re
# from collections import defaultdict
import pipeline as p

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [ ]:
# team_dict = p.open_pkl('team_dictionary.pkl')
# url_team_dict = p.open_pkl('url_team_dict.pkl') 
# elo_stats_team_dict = p.open_pkl('elo_stats_team_dict.pkl')

### Manually create full name team dictionary

In [3]:
# Full team name --> Stats Boxscore abbrev. Includes all teams over years 2008-2018
full_name_team_dict = {'Arizona Cardinals': 'ARI',
 'Atlanta Falcons': 'ATL',
 'Baltimore Ravens': 'BAL',
 'Buffalo Bills': 'BUF',
 'Carolina Panthers': 'CAR',
 'Chicago Bears': 'CHI',
 'Cincinnati Bengals': 'CIN',
 'Cleveland Browns': 'CLE',
 'Dallas Cowboys': 'DAL',
 'Denver Broncos': 'DEN',
 'Detroit Lions': 'DET',
 'Green Bay Packers': 'GNB',
 'Houston Texans': 'HOU',
 'Indianapolis Colts': 'IND',
 'Jacksonville Jaguars': 'JAX',
 'Kansas City Chiefs': 'KAN',
 'Los Angeles Chargers': 'LAC',
 'Los Angeles Rams': 'LAR',
 'Miami Dolphins': 'MIA',
 'Minnesota Vikings': 'MIN',
 'New England Patriots': 'NWE',
 'New Orleans Saints': 'NOR',
 'New York Giants': 'NYG',
 'New York Jets': 'NYJ',
 'Oakland Raiders': 'OAK',
 'Philadelphia Eagles': 'PHI',
 'Pittsburgh Steelers': 'PIT',
 'San Diego Chargers' : 'SDG',
 'San Francisco 49ers': 'SFO',
 'St. Louis Rams': 'STL',
 'Seattle Seahawks': 'SEA',
 'Tampa Bay Buccaneers': 'TAM',
 'Tennessee Titans': 'TEN',
 'Washington Redskins': 'WAS'}

In [18]:
p.pkl_this('full_name_team_dict.pkl', full_name_team_dict)

### Dctionary for Stats: all years, different abbrevs --> one abbrev for all years
this only applies to teams that moved

In [4]:
moved_teams_corr_dict = {
    'LAR':'RAM',
    'STL':'RAM',
    'LAC':'CHG',
    'SDG':'CHG'
}

In [19]:
p.pkl_this('moved_teams_corr_dict.pkl', moved_teams_corr_dict)

### Elo --> Stats dictionary

In [5]:
stats_teams = [x for x in full_name_team_dict.values()]

In [6]:
elo_df = p.open_pkl('elo_df.pkl')
elo_teams = elo_df['team1'].unique()
elo_teams = [x for x in elo_teams]

In [7]:
same = list(set(stats_teams).intersection(elo_teams))
diff = np.setdiff1d(stats_teams, elo_teams)

In [8]:
same

['MIN',
 'IND',
 'PHI',
 'NYJ',
 'ATL',
 'ARI',
 'CIN',
 'BAL',
 'CLE',
 'DET',
 'NYG',
 'TEN',
 'LAC',
 'CAR',
 'PIT',
 'OAK',
 'CHI',
 'DEN',
 'JAX',
 'MIA',
 'HOU',
 'SEA',
 'DAL',
 'BUF',
 'LAR']

In [9]:
diff

array(['GNB', 'KAN', 'NOR', 'NWE', 'SDG', 'SFO', 'STL', 'TAM', 'WAS'],
      dtype='<U3')

In [10]:
elo_subtract = [x for x in elo_teams if x not in same]
elo_subtract.sort()

elo_subtract

['GB', 'KC', 'NE', 'NO', 'SF', 'TB', 'WSH']

In [11]:
# elo has converted San Diego and St. Louis to LAC and LAR, so can remove them from diff
i1 = np.where(diff=='SDG')
i2 = np.where(diff=='STL')
diff = np.delete(diff, [i1,i2])
diff

array(['GNB', 'KAN', 'NOR', 'NWE', 'SFO', 'TAM', 'WAS'],
      dtype='<U3')

In [12]:
# Manually create 'diff_elos' 
# this list is the Elo team abbreviation for teams whose stats (boxscore) abbreviations are different)
# order is that in 'diff' list
diff_elos = ['GB','KC','NO','NE','SF','TB','WSH']

In [13]:
same_elos = [x for x in same]

In [14]:
# Can't just "add" the lists because the dtype of diff is weird ('<U3'). So, a work-around:

stats_teams2 = same
stats_teams2.extend(diff)

elo_teams2 = same_elos
elo_teams2.extend(diff_elos)

elo_stats_team_dict = {}

for i in range(len(stats_teams2)):
    elo_stats_team_dict[elo_teams2[i]] = stats_teams2[i]

In [15]:
elo_stats_team_dict

{'ARI': 'ARI',
 'ATL': 'ATL',
 'BAL': 'BAL',
 'BUF': 'BUF',
 'CAR': 'CAR',
 'CHI': 'CHI',
 'CIN': 'CIN',
 'CLE': 'CLE',
 'DAL': 'DAL',
 'DEN': 'DEN',
 'DET': 'DET',
 'GB': 'GNB',
 'HOU': 'HOU',
 'IND': 'IND',
 'JAX': 'JAX',
 'KC': 'KAN',
 'LAC': 'LAC',
 'LAR': 'LAR',
 'MIA': 'MIA',
 'MIN': 'MIN',
 'NE': 'NWE',
 'NO': 'NOR',
 'NYG': 'NYG',
 'NYJ': 'NYJ',
 'OAK': 'OAK',
 'PHI': 'PHI',
 'PIT': 'PIT',
 'SEA': 'SEA',
 'SF': 'SFO',
 'TB': 'TAM',
 'TEN': 'TEN',
 'WSH': 'WAS'}

In [16]:
# Fix the Chargers & the Rams to my liking
elo_stats_team_dict['LAR'] = 'RAM'
elo_stats_team_dict['LAC'] = 'CHG'

In [17]:
p.pkl_this('elo_stats_team_dict.pkl', elo_stats_team_dict)